## Release of foreman_remote_execution gem

### Prerequisites
- Python 3

If you use Visual Studio Code, you should be all set.
- Jupyter: pip install jupyter

### Requirements
- push access to https://github.com/theforeman/foreman_remote_execution
- push access to rubygems.org for foreman_remote_execution
- sudo yum install python3-slugify asciidoc
- install Transifex client: curl -o- https://raw.githubusercontent.com/transifex/cli/master/install.sh | bash
- ensure neither the `git push` or `gem push` don't require interractive auth. If you can't use api key or ssh key to auth skip these steps and run them from the shell manually 
- to push translations you need an account on Transifex (https://docs.transifex.com/client/client-configuration#~/-transifexrc)

### Release process
Beware of the security risks envolved - access to jupyter notebook means unrestricted access to the shell as the user who runs the Jupyter.
- If you use Visual Studio Code, simply open this file and proceed with the steps
- If you don't use Visual Studio Code, run the server:
  - jupyter notebook rel-eng/gem_release.ipynb (within virtual machine, use jupyter notebook --ip 0.0.0.0 rel-eng/gem_release.ipynb)
- Follow the steps with `<Shift>+<Enter>` or `<Ctrl>+<Enter>,<Down>`
- If anything fails, fix it and re-run the step if applicable

### Release settings

In [ ]:
%cd ..


### Update the following notebook settings

In [ ]:
NEW_VERSION = '12.0.0'
LAST_VERSION = '11.1.1'
NEXT_FUTURE_VERSION = '12.1.0'
MAJOR_RELEASE = True # Change to False if you're releasing z version for X.Y.z
STABLE_BRANCH = '12.0.z'
GIT_REMOTE_UPSTREAM = 'upstream'
WORK_BRANCH = 'master' if MAJOR_RELEASE else STABLE_BRANCH
CHERRY_PICKS = [] # Comma-separated strings of hashes of commits needed for CP


### Ensure the repo is up to date

In [ ]:
! git checkout {WORK_BRANCH}


In [ ]:
! git fetch {GIT_REMOTE_UPSTREAM}


In [ ]:
! git rebase {GIT_REMOTE_UPSTREAM}/{WORK_BRANCH}


### Cherry picks for minor release

In [ ]:
if not MAJOR_RELEASE:
    for cp in CHERRY_PICKS:
        ! git cherry-pick -x {cp}


### Update release related stuff

In [ ]:
! sed -i 's/VERSION = .*/VERSION = "{NEW_VERSION}".freeze/' lib/foreman_remote_execution/version.rb


In [ ]:
# Parse git changelog
from IPython.display import Markdown as md
from subprocess import check_output
from shlex import split
import re

def format_log_entry(entry):
    issues = re.findall(r'[^(]#([0-9]+)', entry)
    entry = re.sub(r'([fF]ixes|[rR]efs)[^-]*-\s*(.*)', r'\2', entry)
    entry = '* ' + entry.capitalize()
    entry = re.sub(r'\(#([0-9]+)\)', r'([PR #\1](https://github.com/theforeman/foreman_remote_execution/pull/\1))', entry)
    for i in issues:
        referenced_issues.append(i)
        entry = entry + ', [#%s](http://projects.theforeman.org/issues/%s)' % (i, i)
    return entry

def skip(entry):
    if re.match(r'Merge pull', entry) or \
      re.match(r'^i18n', entry) or \
      re.match(r'^Bump version to', entry):
        return True
    else:
        return False
referenced_issues = []
git_log_cmd = 'git log --pretty=format:"%%s" v%s..HEAD' % LAST_VERSION
log = check_output(split(git_log_cmd)).decode('utf8').split('\n')
change_log = [format_log_entry(e) for e in log if not skip(e)]
md('\n'.join(change_log))


In [ ]:
# Write release notes
from datetime import datetime
import fileinput
import sys

fh = fileinput.input('release_notes.md', inplace=True)
for line in fh:
    print(line.rstrip())
    if re.match(r'========', line):
        print('### %s (%s)' % (NEW_VERSION, datetime.today().strftime('%Y-%m-%d')))
        for entry in change_log:
            print(entry)
        print('')
fh.close()


# Manual step: Update deps in the gemspec if neccessary

### Check what is going to be commited

In [ ]:
! git add -u
! git status


In [ ]:
! git diff --cached


### Commit changes

In [ ]:
! git commit -m "Bump version to {NEW_VERSION}"


### Update translations

In [ ]:
if MAJOR_RELEASE:
    ! make -C locale/ tx-update


In [ ]:
if MAJOR_RELEASE:
    %cd ../foreman
    ! bundle install
    ! bundle exec rake plugin:gettext[foreman_remote_execution]
    ! bundle exec rake plugin:po_to_json[foreman_remote_execution]
    %cd ../foreman_remote_execution


In [ ]:
if MAJOR_RELEASE:
    ! make -C locale/ mo-files


### Tag new version

In [ ]:
! git tag v{NEW_VERSION}


### Prepare stable branch for major release

In [ ]:
if MAJOR_RELEASE:
    ! git checkout -b {STABLE_BRANCH}
    ! git push {GIT_REMOTE_UPSTREAM} {STABLE_BRANCH}
    ! git checkout {WORK_BRANCH}


### Build the gem

In [ ]:
! bundle exec rake build


In [ ]:
! gem push pkg/foreman_remote_execution-{NEW_VERSION}.gem


### PUSH the changes upstream If everything is correct

In [ ]:
! git push {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}


In [ ]:
! git push --tags {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}


#### Now the new release is in upstream repo

### Some manual steps follow to improve the UX

#### New relase on GitHub

Copy the following changelog lines to the description in form on link below
The release title is the new version.

In [ ]:
print('\n')
print('\n'.join(change_log))
print('\n\nhttps://github.com/theforeman/foreman_remote_execution/releases/new?tag=v%s' % NEW_VERSION)


In [ ]:
from IPython.display import Markdown as md

md('### Create new foreman_remote_execution release in Redmine \n' + \
    '<a href="https://projects.theforeman.org/projects/foreman_remote_execution/versions/new" target="_blank">https://projects.theforeman.org/projects/foreman_remote_execution/versions/new</a>\n\n' + \
    'Set name to foreman_remote_execution-%s' % (NEXT_FUTURE_VERSION if MAJOR_RELEASE else NEW_VERSION))


In [ ]:
if not MAJOR_RELEASE:
    print('Set fixed in versions to %s in following issues:' % NEW_VERSION)
    for i in referenced_issues:
        print('- https://projects.theforeman.org/issues/%s' % i)


## Congratulations

Release is public now. Please, don't forget to do packaging (https://github.com/theforeman/foreman-packaging).